## 강화학습(Reinforcement Learning)이란?
위키피디아에 따르면 강화학습의 정의는 다음과 같다.
>어떤 환경 안에서 정의된 에이전트가 현재의 상태를 인식하여, 선택 가능한 행동 중 보상을 최대화하는 행동 혹은 행동 순서를 선택하는 방법

강화학습은 **정답이나 잘못된 선택에 대한 정정이 주어지지 않는** 특징이 있다. 그렇기에 MDP(Markov Decision Process)를 정의해 강화학습 문제를 푼다.

### 예시
- [AlphaGo](https://deepmind.com/research/case-studies/alphago-the-story-so-far)
- [AlphaStar](https://deepmind.com/research/publications/AlphaStar-Grandmaster-level-in-StarCraft-II-using-multi-agent-reinforcement-learning)

## MDP(Markov Decision Process)
MDP는 다음 구성요소를 가진다.
- 상태 (State)
- 행동 (Action)
- 보상 (Reward)
- 상태 변환 확률 (State Transition Probability)
- 감가율 (Discount Factor)

강화학습 문제를 풀 땐 MDP를 잘 정의하는 게 우선이다. MDP의 각 요소가 무엇인지는 이제 직접 환경을 보며 알아보자.

## OpenAI Gym
강화학습엔 환경(environment)이 필요하다. OpenAI Gym는  다양한 강화학습 환경을 제공한다.

이제 OpenAI Gym을 설치해 본격적으로 강화학습을 실습해보자.


In [0]:
# 필요한 패키지를 설치해준다.
!cat requirements.txt
!echo '----------------------------------'
!pip install -r requirements.txt

gym
numpy
matplotlib
----------------------------------


## Frozen Lake
처음으로 실습할 환경은 ***Frozen Lake***다. Frozen Lake는 에이전트가 구멍을 피해 얼어있는 강을 건너는 환경이다.

판의 크기는 4 by 4로 총 16칸이며, 에이전트는 상하좌우로 한 칸 이동할 수 있다.

In [0]:
# 다음 코드를 통해 FrozenLake 환경을 불러올 수 있다.
import gym

env = gym.make("FrozenLake-v0")

In [0]:
# 환경에 관한 정보는 다음과 같이 얻을 수 있다.

print('Observation Space:', env.observation_space)
print('Action Space:', env.action_space)

env.render()

Observation Space: Discrete(16)
Action Space: Discrete(4)

SFFF
FHFH
FFFH
HFFG


In [0]:
# episode를 시작하기 전엔 reset 메소드를 호출해야한다.
# reset 메소드는 state를 반환한다.
state = env.reset()
print(state)

# step 메소드를 이용해 한 스텝 진행할 수 있다.
# step 메소드는 새로운 state, 보상, 끝났는지 여부, 기타 정보를 반환한다.
next_state, reward, done, _ = env.step(env.action_space.sample())
print('next_state:', next_state)
print('reward:', reward)
print('done:', done)

0
next_state: 1
reward: 0.0
done: False


In [0]:
# 지금까지 배운 내용으로 무작위로 행동하는 에이전트를 만들어보자.

# 환경을 만든다
env = gym.make('FrozenLake-v0')

# Episode를 시작하기 전엔 반드시 reset 메소드를 호출해야한다.
state = env.reset()

while True:
  # 환경을 출력한다.
  env.render()

  # Action space에서 무작위 행동을 가져온다.
  action = env.action_space.sample()

  # 생성한 action을 바탕으로 한 스텝 진행한다.
  new_state, reward, done, _ = env.step(action)

  # Episode가 끝났다면 반복문을 빠져나간다.
  if done:
    print(reward)
    break

  # state에 새로운 state를 할당한다. (지금은 의미 없음)
  state = new_state



SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
0.0


## Q Learning
어떤 상황 *s*에서 어떤 행동 *a*의 가치를 Q(s, a)라 하자. 그렇다면 가장 좋은 행동은 Q(s, a)가 가장 큰 *a*이다.

그렇다면 Q 값은 어떻게 구할까? 보상(reward)를 바탕으로 Q 값을 갱신하면 된다.